<a href="https://colab.research.google.com/github/ichekhovskikh/skip-thought-vectors/blob/master/skip_thoughts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка

Установим флаг floatX=float32, чтобы избежать ошибок типа TypeError.

In [0]:
!echo -e "\n[global]\nfloatX=float32\n" >> ~/.theanorc

Необходимо подлючить Google Drive, где по пути skip-thoughts-master/training располагается библиотека для работы со Skip Throughts Vectors. Папку training можно скачать из репозитория https://github.com/ichekhovskikh/skip-thought-vectors

In [0]:
from google.colab import drive

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Сделать импорт библиотеки

In [0]:
import importlib.util
import sys

sys.path.append('/content/drive/My Drive/skip-thoughts-master/training')
import vocab
import train
import tools

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
!pip install pymorphy2[fast]

import gensim
from gensim.models import Word2Vec, KeyedVectors
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk import tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.3MB/s 
     |████████████████████████████████| 378kB 53.2MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=862191 sha256=aeebc0b1daa773b3ec116a29fb2e6c80743a256b9c8f808bd71688741b15f363
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/data.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            sentences = tokenize.sent_tokenize(article['text'], 'russian')
            normalized_sentences = []
            for sentence in sentences:
                normalized_sentences.append(' '.join(advanced_preprocess(sentence)))
            yield normalized_sentences

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Загружаем корпуса:

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path))

Давайте посмотрим на учебный и тестовый корпуса:

In [0]:
print(train_corpus[:2])

[['общественный жизнь экономика рынок каждый готовить блин сковорода антипригарный покра тие знать важный температура приготовление который зависеть цвета форма изделие домаш условие некоторый колебание цвет толщина форма вполне приемлемый промышленный произа водство блин сложный процесс неоднородность готовый продукт становиться больший пра пятствие успешный реали зации торговый сеть новое технический решение стабилизация качество промышленный ный производство блин разрабо таль специалист шведский фирма formcook промышленный печь термообработка самый разно образный продукт мясо птица рыба овощ пять год поставлять россия компания агро надёжный высокопроиза водительный оборудование успеть завоевать авторитет российский специалист мясной рыбный молочный промышленность сис тема общепит инженер конструктор фирма formcook промышленный производство блин год спустя запустить эксплуата цию один инновационный система приготовление пищевой промышленность течение десяток год обходиться старомодны

In [0]:
print(test_corpus[:2])

[['язык ассемблер машинный ориентировать язык низкий уровень', 'команда прямо соответствовать отдельный команда машина также предоставлять дополнительный возможность облегчение такой макрокоманда выражение средство обеспечение модульность программа', 'рассматриваться автокод расширить конструкция язык высокий уровень', 'являться существенно платформо зависимый', 'язык ассемблер различный аппаратный платформа несовместимый хотя мочь целое подобный', 'русский язык именоваться просто ассемблер типичный выражение тип писать программа ассемблер строго говорить неверный ассемблер именоваться утилит трансляция программа язык ассемблер объектный код компьютер'], ['перитонит развиваться вследствие бактериальный инфицирование брюшной полость', 'подавлять большинство пациент перитонит обнаруживать различный сочетание аэробный анаэробный микроорганизм', 'отсутствие рост микрофлора стандартный исследование условие гнойный каловый перитонит свидетельствовать лишь высокий вероятность доминировать ана

Создайте три пустых файла в Google Drive для сохранения в них словаря, модели Word2Vec, модели Skip Throughts Vectors

In [0]:
path_to_dictionary = '/content/drive/My Drive/sample_data/vocab.pkl'
path_to_word2vec = '/content/drive/My Drive/sample_data/word2vec.bin'
path_to_model = '/content/drive/My Drive/sample_data/model.npz'

# Обучение модели
## Создание словаря
Состовим словарь из тренеровочного корпуса предложений и сохраним его по пути path_to_dictionary для дальнейшего использования

Объединение всех предложений в один список

In [0]:
def flatten(text):
    return [item for sublist in text for item in sublist]

Создание и сохранение словаря

In [0]:
sentenses = flatten(train_corpus)
worddict, wordcount = vocab.build_dictionary(sentenses)
vocab.save_dictionary(worddict, wordcount, path_to_dictionary)

 ## Создание модели Word2Vec
Составим модель Word2Vec из тренеровочного корпуса предложений и сохраним его по пути path_to_word2vec для дальнейшего использования

Разделить предложение на слова

In [0]:
def word_tokenize(text):
    flat_text = flatten(text)
    return [item.split(' ') for item in flat_text]

Создание и сохранение модели Word2Vec

In [0]:
#@title Укажите параметры обучения модели слов:
vector_size = 80 #@param
window =  5 #@param
epochs =  100 #@param
min_count = 2 #@param
alpha = 0.001 #@param
learning_method = "skip-gram" #@param ["skip-gram", "CBOW"] {type:"raw"}

sg = 1
if (learning_method == "CBOW"):
    sg = 0

In [0]:
words = word_tokenize(train_corpus)
model = Word2Vec(
    words, 
    size=vector_size, 
    window=window, 
    min_count=min_count, 
    alpha=alpha,
    sg=sg,
    iter=epochs,
    workers=4)

model.wv.save_word2vec_format(path_to_word2vec, binary=True)

# Создание модели Skip Throughts Vectors
Создадим модель Skip Throughts Vectors для преобрахования текстов в вектора.
Сохраним ее в Google Drive по пути path_to_model. Параметры обучения:
- dim_word: размерность вектора слов в RNN сети
- max_epochs: количество эпох обучения
- displayFreq: отображение прогресса после такого количества эпох
- learning_rate: коэффициент обучения
- n_words: размерность словаря
- maxlen_w: максимальное количество слов в предложении, если ниже заданной - игнорировать
- saveto: путь для сохранения модели
- dictionary: путь к словарю
- saveFreq: частота сохранения модели в эпохах

In [0]:
#@title Укажите параметры обучения модели предложений:
max_epochs = 100 #@param
neurons_count = 2400 #@param
learning_rate = 0.01 #@param

In [0]:
sentenses = flatten(train_corpus)
%time train.trainer(
    sentenses, 
    n_words = len(worddict) + 2, # + 2 (EOF, UNK)
    dim=neurons_count,
    learning_rate=learning_rate,
    max_epochs=max_epochs,
    dictionary = path_to_dictionary,
    saveto = path_to_model,
    saveFreq = 1)

Загрузка обученной модели

In [0]:
model = tools.load_model(
    path_to_model = path_to_model,
    path_to_dictionary = path_to_dictionary,
    path_to_word2vec = path_to_word2vec)

# Поиск похожих текстов на основе обученной модели

Обратная сглаженная частота (вес слова), где
- a: гиперпараметр, который обычно равен 0.001 (конфигурация, внешняя по отношению к модели, значение которой невозможно оценить по данным):

In [0]:
a = 0.001

def sif(word, sentence):
    frequency = sentence.count(word) / len(sentence)
    return a / (a + frequency) 
    

Получить вектор текста:

In [0]:
def paragraph_vector(model, paragraph):
    sentence_vectors = tools.encode(model, paragraph)
    paragraph_vector = []
    vector_count = 0
    for sentence_vector in sentence_vectors:
        if not paragraph_vector:
            paragraph_vector = sentence_vector
        else: 
            paragraph_vector = [x + y for x, y in zip(paragraph_vector, sentence_vector)]
            vector_count += 1
    if vector_count != 0:
        paragraph_vector = [x / vector_count for x in paragraph_vector]
    return paragraph_vector

Определение косинусной близости двух векторов:

In [0]:
def cosine_similarity(first_vector, second_vector):
    numerator = sum([x * y for x, y in zip(first_vector, second_vector)])
    first_norm = pow(sum([x * x for x in first_vector]), 0.5)
    second_norm = pow(sum([x * x for x in second_vector]), 0.5)
    return numerator / (first_norm * second_norm)

Поиск похожих текстов из корпуса

In [0]:
def sims(model, corpus, corpus_path, paragraph, count = 4):
    result = []
    compared_vector = paragraph_vector(model, paragraph)
    for index, text in enumerate(corpus):
        text_vector = paragraph_vector(model, text)
        similarity = cosine_similarity(compared_vector, text_vector)
        result.append([similarity, get_article_text_by_index(index, corpus_path)])
    return reversed(sorted(result,  key=lambda x: x[0]))[:count]

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)
doc = train_corpus[doc_id]
sims = sims(model, train_corpus, train_path, doc)
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, train_path)))
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))


# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
doc_id = random.randint(0, len(test_corpus) - 1)
doc = test_corpus[doc_id]
sims = sims(model, train_corpus, train_path, doc)
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, test_path)))
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))

Тестирование на малых данных для выявления ошибок в ходе написания кода

In [0]:
text = ["первое предложение", "второе предложение", "третье предложение", "что-то еще предложение"]
text_split_on_words = [['первое', 'предложение'], ['второе', 'предложение'], ['третье', 'предложение'], ['что-то', 'еще', 'предложение']]

worddict, wordcount = vocab.build_dictionary(text)
vocab.save_dictionary(worddict, wordcount, path_to_dictionary)

model = Word2Vec(text_split_on_words, size=300, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format(path_to_word2vec, binary=True)

train.trainer(
    text, 
    n_words=len(worddict) + 2,
    dictionary=path_to_dictionary,
    saveto=path_to_model,
    saveFreq=1)

model = tools.load_model(
    path_to_model = path_to_model,
    path_to_dictionary = path_to_dictionary',
    path_to_word2vec = path_to_word2vec)

vectors = tools.encode(model, text)
print(cosine_similarity(vectors[0], vectors[1]))

# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read()


In [0]:
normalized_text = advanced_preprocess(article_text)
sims = sims(model, train_corpus, train_path, normalized_text)

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
num = 0
for similarity, text in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, text))